In [7]:
from PIL import Image 
import os
import csv
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch
import pandas as pd

In [8]:
class ImageComparisonModel(nn.Module):
    def __init__(self):
        super(ImageComparisonModel, self).__init__()
        
        # Stream 1 for the first image modality
        self.stream1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Stream 2 for the second image modality
        self.stream2 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Fusion and decision mechanism
        self.comparison = nn.Sequential(
            nn.Linear(32*32*128*2, 1024),  # Assuming images are 224x224 size, adjust if different
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1)  # Binary classification
        )
        
    def forward(self, x1, x2):
        x1 = self.stream1(x1)
        x2 = self.stream2(x2)
        
        # Flatten the features from both streams
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        
        # Concatenate the features
        x = torch.cat((x1, x2), dim=1)
        
        # Pass through the comparison mechanism
        x = self.comparison(x)
        return x

In [9]:
# 결과를 저장할 데이터프레임 생성
results_df = pd.DataFrame(columns=['Image1', 'Image2', 'Prediction'])

# 이미지를 불러오고 전처리
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # 이미지 크기를 64x64로 조정
    transforms.ToTensor(),         # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

In [10]:
# 모델 불러오기
model = ImageComparisonModel() # 모델 클래스를 불러올 때와 같은 방식으로 모델을 정의해야 합니다.
model.load_state_dict(torch.load('Check_Model.pth'))
model.eval()  # 모델을 평가 모드로 설정

ImageComparisonModel(
  (stream1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (stream2): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (comparison): Sequential(
    (0): Linear(in_features=262144, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Li

In [15]:
# 이미지가 있는 두 개의 폴더 경로 설정
folder1_path = "test_data/chemdraw_test_smlies"
folder2_path = "test_data/chemdraw_test_smlies_wrong"

# 이미지 폴더 내의 파일 목록 가져오기
folder1_files = os.listdir(folder1_path)
folder2_files = os.listdir(folder2_path)

In [16]:
# 결과를 저장할 CSV 파일 경로 설정
output_csv_path = "result_correct.csv"

# 결과를 저장할 리스트 초기화
results = []


# 폴더1의 이미지와 폴더2의 이미지를 비교하며 모델 평가
for file1, file2 in zip(folder1_files, folder2_files):
    # 이미지 파일의 경로 설정
    image1_path = os.path.join(folder1_path, file1)
    image2_path = os.path.join(folder2_path, file2)

    # 이미지 불러오기
    image1 = Image.open(image1_path).convert("RGB")
    image2 = Image.open(image2_path).convert("RGB")

    # 전처리 적용
    image1 = transform(image1).unsqueeze(0)  # 배치 차원 추가
    image2 = transform(image2).unsqueeze(0)  # 배치 차원 추가

    # 모델에 이미지 입력하여 예측 수행
    with torch.no_grad():
        output = model(image1, image2)
        predicted_label = torch.sigmoid(output).item()  # 이진 분류에서 확률값으로 변환

    # 결과를 리스트에 저장
    results.append([file1, file2, predicted_label])
    

# 결과를 CSV 파일에 저장
with open(output_csv_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Image1", "Image2", "Predicted_Probability"])
    writer.writerows(results)

print("Results saved to:", output_csv_path)

Results saved to: result_correct.csv
